In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, Attention, DecoderAttnRNN
from train_eval import train_and_eval, inspect_model, count_parameters, summarize_results, plot_single_learning_curve
import importlib
import pickle as pkl 

In [2]:
# parameters and hyperparameters 
SRC_LANG = 'vi'
TARG_LANG = 'en'
SRC_VOCAB_SIZE = 30000
TARG_VOCAB_SIZE = 30000
BATCH_SIZE = 32
SRC_MAX_SENTENCE_LEN = 10 
TARG_MAX_SENTENCE_LEN = 10
NUM_EPOCHS = 2000 
LR = 0.0005
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 300 
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM

In [3]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [4]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, vocab)
limited_data = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 

In [5]:
# create dataloaders 
full_loaders = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
fast_loaders = create_dataloaders(limited_data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [6]:
# define model 

encoder = EncoderRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                     pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderAttnRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                          targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
#                          targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                          pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [7]:
train_and_eval(model=model, full_loaders=full_loaders, fast_loaders=fast_loaders, id2token=vocab[TARG_LANG]['id2token'],
               learning_rate=LR, num_epochs=NUM_EPOCHS, print_intermediate=True, save_checkpoint=True, model_name='test2', 
               lazy_eval=False, lazy_train=True, inspect=True)

Epoch: 0.00, Train Loss: 10.10, Val Loss: 10.23, Train BLEU: 0.28, Val BLEU: 0.29
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: . . . . . . . . . fashion adaptation the the of of of of of even , , of of of of of of defiance , of of of of of of of ingels , , , of of of of of
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: graded , , of of of of of of panicky . . . . . . . . recreation , , of of of of of of rooting rooting of of of of of of of accents , , of of of of of of
Epoch: 1.00, Train Loss: 9.83

Epoch: 10.00, Train Loss: 5.32, Val Loss: 8.31, Train BLEU: 0.28, Val BLEU: 0.50
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: of of of of of of of of of the of , , , , , , , the of of , , , , , , the of , , , , , , , the of of , , , , , ,
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: of of of , , , , , , of of of of of of of of of of of of , , , , , , the of , , , , , , , the of of , , , , , ,
Epoch: 11.00, Train Loss: 5.07, Val Loss: 8.36, Train BLEU: 0.62, Val BLEU: 0.43
Inspecting model on trai

Epoch: 19.00, Train Loss: 4.31, Val Loss: 9.40, Train BLEU: 0.50, Val BLEU: 0.25
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: rachel wrote . . . . . . . like the the the the the the the it to the the the the the the the the like the the the the the the it it to the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to the the the the the the the the when . . . . . . . . to the the the the the the the the like the the the the the the it it to the the the the the the the the
E

Epoch: 28.00, Train Loss: 4.00, Val Loss: 10.13, Train BLEU: 5.16, Val BLEU: 0.26
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: rachel when wrote euphore euphore euphore euphore euphore euphore like the the the the the the the the to to the the the the the the the like the the the the the the the the to to the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: behind to the the the the the the the rachel when of . . . . . . to to the the the the the the the like the the the the t

Epoch: 36.00, Train Loss: 3.75, Val Loss: 10.41, Train BLEU: 6.14, Val BLEU: 0.25
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: rachel when wrote of euphore euphore euphore euphore euphore like the the the the the the the the to to the the the the the the the like the the the the the the the the to to the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to the the the the the the the rachel when wrote of of is is is is to to the the the the the the the like the the the the t

...

Epoch: 1972.00, Train Loss: 0.00, Val Loss: 14.82, Train BLEU: 10.36, Val BLEU: 0.88
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind wrote francisco of . euphore is over over : they pages every molecules it it hundreds the pike intergovernmental community and weight blow the perform our : have big year and because in have looking pike they community and and it smog our have
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the intergovernmental community and weight it smog , in : when when of of o

Epoch: 1980.00, Train Loss: 0.00, Val Loss: 14.83, Train BLEU: 10.36, Val BLEU: 0.88
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind wrote francisco of . euphore is over over : they pages every molecules it it hundreds the pike intergovernmental community and weight blow the perform our : have big year and because in have looking pike they community and and it smog our have
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the intergovernmental community and weight it smog , in : when when of of o

Epoch: 1988.00, Train Loss: 0.00, Val Loss: 14.83, Train BLEU: 10.36, Val BLEU: 0.88
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind wrote francisco of . euphore is over over : they pages every molecules it it hundreds the pike intergovernmental community and weight blow the perform our : have big year and because in have looking pike they community and and it smog our have
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the intergovernmental community and weight it smog , in : when when of of o

Epoch: 1996.00, Train Loss: 0.00, Val Loss: 14.83, Train BLEU: 10.36, Val BLEU: 0.88
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind wrote francisco of . euphore is over over : they pages every molecules it it hundreds the pike intergovernmental community and weight blow the perform our : have big year and because in have looking pike they community and and it smog our have
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the intergovernmental community and weight it smog , in : when when of of o

[{'epoch': 0.0,
  'train_bleu': 0.28102309099910433,
  'train_loss': 10.098424911499023,
  'val_bleu': 0.2863737368333751,
  'val_loss': 10.231797218322754},
 {'epoch': 1.0,
  'train_bleu': 0.6915281505062035,
  'train_loss': 9.830333709716797,
  'val_bleu': 0.5186475558462794,
  'val_loss': 10.1244535446167},
 {'epoch': 2.0,
  'train_bleu': 0.7362458216751531,
  'train_loss': 9.406909942626953,
  'val_bleu': 0.6245391199393742,
  'val_loss': 9.942559242248535},
 {'epoch': 3.0,
  'train_bleu': 0.7182310645114018,
  'train_loss': 8.814777374267578,
  'val_bleu': 0.5083722182766547,
  'val_loss': 9.674050331115723},
 {'epoch': 4.0,
  'train_bleu': 0.7027761990859905,
  'train_loss': 8.166820526123047,
  'val_bleu': 0.5048044735319812,
  'val_loss': 9.361627578735352},
 {'epoch': 5.0,
  'train_bleu': 0.4852369627775136,
  'train_loss': 7.555747985839844,
  'val_bleu': 0.5048044735319812,
  'val_loss': 9.055387496948242},
 {'epoch': 6.0,
  'train_bleu': 0.4624527462692997,
  'train_loss': 